In [1]:
import requests
import pandas as pd

In [2]:
def get_playlist_id(channel_id, api_key):
    playlist_url = "https://www.googleapis.com/youtube/v3/channels?"
    playlist_part = "contentDetails"
    playlist_parameters = {"key": api_key,
                           "part": playlist_part,
                           "id": channel_id}

    playlist_query = requests.get(playlist_url, playlist_parameters)
    playlist_results = playlist_query.json()
    playlist_id = playlist_results["items"][0][playlist_part]["relatedPlaylists"]["uploads"]
    return playlist_id


def get_videos(playlist_id, columns, api_key):
    video_url = "https://www.googleapis.com/youtube/v3/playlistItems?"
    video_part = "snippet"
    max_results = 50  # api supports up to 50
    video_parameters = {"key": api_key,
                        "part": video_part,
                        "playlistId": playlist_id,
                        "maxResults": max_results,
                        "pageToken": ""}

    video_query = requests.get(video_url, video_parameters)
    video_results = video_query.json()
    video_data = []
    for i in range(len(video_results["items"])):
        video_data.append([video_results["items"][i]["snippet"][column] for column in columns])

    while "nextPageToken" in video_results:
        video_parameters["pageToken"] = video_results["nextPageToken"]
        video_query = requests.get(video_url, video_parameters)
        video_results = video_query.json()
        for i in range(len(video_results["items"])):
            video_data.append([video_results["items"][i]["snippet"][column] for column in columns])

    return video_data

In [3]:

arxiv = "UCNIkB2IeJ-6AmZv7bQ1oBYg" #arXiv
dunkey = "UCsvn_Po0SmunchJYOWpOxMg" #dunkey
video_columns = ["title", "description", "publishedAt"]

uploads = get_playlist_id(dunkey, api_key)
video_dataframe = pd.DataFrame(get_videos(uploads, video_columns, api_key), columns=video_columns)
video_dataframe

,title,description,publishedAt
0,Dunkey Halloween Special,Check out our new halloween shirt at https://d...,2018-10-25T20:00:00.000Z
1,Call of Duty 15 : Black Ops 4,This time everyone has built in juggernaut.\n\...,2018-10-21T19:30:01.000Z
2,Assassin's Creed Good Franchise : Odyssey,"I gotta admit though, I didn't see any trailin...",2018-10-10T19:30:00.000Z
3,Hamston Checks Out,This is my first overwatch video.\n\ndunk stor...,2018-10-02T19:30:00.000Z
4,Kingdom Hearts Explained,Here is a quick recap to prepare you for Kingd...,2018-09-24T19:30:00.000Z
5,Mario's Balloon World,Get it ? C-erious. Because his name is C-Rex.\...,2018-09-14T19:30:00.000Z
6,Spider-Man,With big power comes big power.\n\ndunk store ...,2018-09-10T19:30:00.000Z
7,Shenmue (dunkview),Should've been called Shenmoo and instead you ...,2018-09-01T19:30:02.000Z
8,The Majestic World of Dolphins,Take a dive with Morgan Freeman into the fasci...,2018-08-27T19:30:01.000Z
9,Dunkey Clones,"The shroud of the dark side has fallen, begun ...",2018-08-20T19:30:00.000Z
